# Platform Positioning

In [3]:
from __future__ import print_function

In [4]:
import random as random #For random initial platform position
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import geopandas as gpd
from shapely.geometry.polygon import Polygon
import shapely
from geopy import distance # For calculating distance between two points 
import pickle # For importing pickled objects
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from shapely.geometry import Point, LineString #For Platform and 
from mpl_toolkits.basemap import Basemap # Mapping visualisation
from math import tan, isclose, pi #For calculating point on circumference - step out radius

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [5]:
import os
os.getcwd()

'/Users/Pri.balachandran@ibm.com/Desktop/BP/20191106_PlatformWCAnalysis'

# Data Ingestion & Cleaning
Since Well Concept data is created based on the cleaned Target data, the notebook has been structured with Targets, then Well Concepts

##  Target Geometry Data
This has been queried and pickled. The column names refer to the SQL function performed on geometry.
e.g. geom.AsTextZM() AS geom_text_zm

In [6]:
with open('data/0_raw/TEST_TRIN_20191111', 'rb') as f:
    df_target = pickle.load(f)

Drop rows which don't have geometry - since it is simply being used as an example layer

In [7]:
df_target_wgeom = df_target[df_target['geom_STAs_binary'].notnull()]
df_target_wgeom.head()

,target_id,target_name,geom_text_zm,geom_binary_zm,geom_STAs_binary,geom_STAs_Text
0,768,Kapok-TQ65-SPA,MULTILINESTRING ((-60.498999227043505 9.855164...,b'\x01\xed\x03\x00\x00\x01\x00\x00\x00\x01\xea...,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...,MULTILINESTRING ((-60.498999227043505 9.855164...
1,769,Kapok-TQ40-SPA,MULTILINESTRING ((-60.5041507381779 9.90958420...,"b""\x01\xed\x03\x00\x00\x01\x00\x00\x00\x01\xea...","b""\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...",MULTILINESTRING ((-60.5041507381779 9.90958420...
3,771,Kapok-TQ20b-SPA,MULTILINESTRING ((-60.5024436026266 9.91144177...,b'\x01\xed\x03\x00\x00\x01\x00\x00\x00\x01\xea...,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...,MULTILINESTRING ((-60.5024436026266 9.91144177...
7,775,Kapok-TP95L-SPA,MULTILINESTRING ((-60.481304509800545 9.864397...,b'\x01\xed\x03\x00\x00\x01\x00\x00\x00\x01\xea...,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...,MULTILINESTRING ((-60.481304509800545 9.864397...
8,776,Kapok-TP95-REN,MULTILINESTRING ((-60.479890700013996 9.899954...,b'\x01\xed\x03\x00\x00\x01\x00\x00\x00\x01\xea...,b'\x01\x05\x00\x00\x00\x01\x00\x00\x00\x01\x02...,MULTILINESTRING ((-60.479890700013996 9.899954...


#### Transform Target Geometries
Target geometries are read as strings, formatted as shown below. As such, they must be transformed into Shapely Polygons

In [8]:
print("Example geometry string: \n", df_target_wgeom['geom_STAs_Text'][0][:200], "...")

Example geometry string: 
 MULTILINESTRING ((-60.498999227043505 9.8551643192777156, -60.499975832266763 9.8561382533816655, -60.500665679316072 9.8571019706351954, -60.501906636383438 9.8579163060709938, -60.50271592235876 9.8 ...


In [9]:
def TransformMLStoPolygon(str_mlstring):
    '''Transforms a multilinestring string into a shapely Polygon. X,Y coordinate pairs are extracted from the string.
    Function is likely to fail if presented with real-data, since we have assumed input as a single multilinestring.
    '''
    
    #Remove string pre and post fix
    str_values = str_mlstring.replace("MULTILINESTRING ((", "").replace("))", "")

    #BUG FIX - This line should not be necessary - there were some extra brackets which needed handling
    str_values = str_values.replace("(","").replace(")", "")

    #Split the string into a list of xy pairs
    lst_xystring = str_values.split(", ")

    #Split each xy pair into a tuple, of type float
    lst_tuples = [tuple(map(float,x.split(" "))) for x in lst_xystring]
    
    return Polygon(lst_tuples)

In [37]:
#Transform geometry column into shapely polygon
df_target_wgeom.loc[:,'polygon'] = df_target_wgeom['geom_STAs_Text'].map(lambda x: TransformMLStoPolygon(x))

#Create geoSeries
#gs_targPoly = gpd.GeoSeries(df_target_wgeom.loc[:,'polygon'])
gdf_targPoly = gpd.GeoDataFrame(df_target_wgeom, geometry='polygon')

print(gs_targPoly.head())

0    POLYGON ((-60.49899922704351 9.855164319277716...
1    POLYGON ((-60.5041507381779 9.909584204961554,...
3    POLYGON ((-60.5024436026266 9.911441778228152,...
7    POLYGON ((-60.48130450980054 9.864397257979865...
8    POLYGON ((-60.479890700014 9.899954964645545, ...
Name: polygon, dtype: object


## Well Concept Data
As no suitable test data exists, dummy data is generated. There is a separate dummdatacreator.py file containing this work.

Rather than have Well Concepts randomly placed anywhere, we restrict it to only be within Target Polygon space

In [11]:
l = []

#BUG FIX- Some polygons are invalid since first and last points are the same - they must not be.
for p in gs_targPoly.values:
    if p.is_valid == True:
        l.append(p)
        
# Union of all polygons
targ_poly_union = shapely.ops.cascaded_union(l)

In [12]:
from dummydatacreator import  GenerateWellConcept # This is not supported in Azure notebooks

In [13]:
def UpdateWellConceptData(n_concepts = 30, lon_min = -60.34, lon_max = -60.32, \
                          lat_min = 10.04, lat_max = 10.08):
    '''
    
    
    '''
    #Generate a polygon bounding box from the min and max of lon and lat
    bbox = Polygon([[lon_min,lat_min], [lon_min,lat_max], [lon_max, lat_max], [lon_max, lat_min]])
    
    # Get all polygons within bbx
    bbox.intersection(targ_poly_union)
    
    #Generate Well Concepts
    WellConcepts = [GenerateWellConcept(bbox)  for i in range(n_concepts)]

    return list(zip(*WellConcepts))

In [14]:
WC_name, WC_points, WC_resource = UpdateWellConceptData()

print("Well Concepts Generated:\n",'\n'.join('Name: {}, Points: {}, Resource: {}'.format(WC_name[k], WC_points[k], WC_resource[k]) for k in range(5)))

Well Concepts Generated:
 Name: Sd-15, Points: [-60.33351767141367, 10.05430191841169], Resource: 26.549341075216105
Name: P-2, Points: [-60.33456813893818, 10.077739147729297], Resource: 9.46317512572618
Name: C-4, Points: [-60.333186911657464, 10.055722702703775], Resource: 33.0319798962172
Name: G-16, Points: [-60.33825744737537, 10.056820233824618], Resource: 89.28065982724307
Name: D-22, Points: [-60.33207863969523, 10.062001227173601], Resource: 182.09112452059074


# Mapping
In order to construct the map containing Targets, Well Concepts, Platforms, some functions are required:


### Platform Step-Out Radius Functions
As there is no method for deriving the circumference around a point in EPSG:4326, functions are required to calculate this.
- A function to find a single point on the circumference (within a given tolerance)
- A function to find multiple points all around the circumference, sufficient for plotting the circle

In [15]:
def GetXYatCircumference(centre, desired_radius, theta, tolerance = 0.00001):
    '''Calculates the X,Y coordinates of a point on the circumference of a circle, 
    with a given desired radius and angle to the vertical.
    
    Theta in radians
    '''
    
    #Initialise x,y coordinates
    x = centre[0]
    y = centre[1]

    
    #Set initial increment values
        #If we are working in the lower two quadrants, i.e. where y will be less than centre y
    if theta > pi/2 and theta < (3*pi)/2:
        y_increment=-0.01
    else:
        y_increment=0.01

        
    #Loop until the radius of the point and centre, is close to the desired radius
    while isclose(distance.distance(centre, [x,y]).kilometers,  desired_radius ,rel_tol=0.00001) == False:

        # Add the increment whilst the distance is less than desired
        while distance.distance(centre, [x,y]) < desired_radius:
            x_increment = y_increment * tan(theta)
            x += x_increment
            y += y_increment

        # Subtract the increment whilst the distance is more than desired
        while distance.distance(centre, [x,y]) > desired_radius:
            x_increment = y_increment * tan(theta)
            x -= x_increment
            y -= y_increment

        #Reduce the size of the increment to increase precision  
        y_increment = y_increment/4

    return [x,y]

In [16]:
def GetCircumference(centre, radius, n_points, tolerance = 0.00001):
    '''Calculates x,y coordinates of a specified number points on the circumference of a 
    circle with a specified centre    
    
    NB: n_points must odd, to ensure we do not get a trig erro
    '''
    assert n_points % 2 != 0, "n_points must not be an even number - gives a trig error"

    # The angle between each radius        
    theta_increment = (2*pi)/n_points

    # The angles of all the radii we will use
    thetas = [theta_increment * i for i in range(n_points)]
    
    # Get a list of all calculated XYs on the circumference
    xys = [GetXYatCircumference(centre, radius,theta_i,tolerance) for theta_i in  thetas]
    
    # Create a full 'loop' so the first and last points are the same - Polygon
    xys.append(xys[0])
    
    return xys

GetCircumference([0,0], 2, 3)

[[0.0, 0.01796615600585938],
 [0.015637707705740515, -0.009028434753417969],
 [-0.015637707705740488, -0.009028434753417969],
 [0.0, 0.01796615600585938]]

### Well Concept Relative Position
i.e. Are Well Concepts within specified radii around a point?

In [17]:
def CalculateRankOfWCs(WC, plat, radii):
    '''Well Concepts are ranked according to binned distance from platform. i.e. 
    Given input radii [0,2,5], the function will return a list of rankings where:
    0 indicates Well Concept is more than 5 km from platform
    1 indictates Well Concept is between 2 and 5 km from platform... etc
    '''
    #Radii must be ordered, i.e. [0,2,5]
    radii = np.sort(radii)
    
    num_radii = len(radii)

    #calculate distances of all well concepts from platform 
    distances = [distance.distance(wc_i, plat) for wc_i in WC] #Karney 2013 distance
    
    distances = np.reshape(distances,[-1])

    
    #Reverse ranking - where 0 is closest to platform
    revranks = np.digitize(distances, radii, right = False) #i.e. includes left bound = 0

    # Ranking - where 0 is furthest from platform
    ranking = [num_radii - revrank for revrank in revranks]
    
    return ranking

In [18]:
dct_colors = {0: "#808080",
          1:"#FF8000",
          2:"#00CC00"}

In [19]:
def GetWCColors(WC_Points, plat_x, plat_y,radii):
    '''
    Given the rank of Well Concepts, this derives the appropriate color for them
    '''
    #Subset the WCs to only those which are on the creaming curve
    WC_rank = CalculateRankOfWCs([(a[0], a[1]) for a in WC_points],(plat_x,plat_y),radii)

    #Construct dataframe of all well concepts
    df_WC = pd.DataFrame({"Well Concept": WC_name,"Resource": WC_resource, "Rank":WC_rank})
    
    return df_WC['Rank'].map(lambda x: dct_colors[x])    

## Map Construction

In [20]:
from shapely.ops import transform
from descartes import PolygonPatch
from matplotlib.collections import PatchCollection

In [21]:
def GenerateMap(WC_Point : Point,WC_name, WC_res: np.array,WC_col, 
                     plat_lon, plat_lat, plat_name, 
                     lon_min, lon_max, lat_min, lat_max,
                     radii,
                     figsize):
    '''Creates a basemap/matplotlib map of Target Polygons, Well Concepts, Platform and Platform step-out radii

    '''

    #Split Well Concept points to separate lat long
    WC_lon, WC_lat=list(zip(*WC_Point))

    
    fig, ax = plt.subplots(figsize=figsize)

    water = 'lightskyblue'
    earth = 'cornsilk'
    
    #Initialise basemap
    mm = Basemap(
        resolution='i',
        epsg=4326,
        llcrnrlon=lon_min,
        llcrnrlat=lat_min,
        urcrnrlon=lon_max,
        urcrnrlat=lat_max,
        lon_0=(lon_min+lon_max)/2,
        lat_0=(lat_min+lat_max)/2)
        

    bound= mm.drawmapboundary(fill_color=water)
    countries = mm.drawcountries()
    
    
    
    #x axis labels
    merid = mm.drawmeridians(
        np.arange(lon_min, lon_max, 0.02), 
        labels=[False, False, False, True])
    
    #y axis labels
    parall = mm.drawparallels(
        np.arange(lat_min, lat_max, 0.01), 
        labels=[True, True, False, False])
    
    
    
    #Target Polygon Visualisation
    patches = []
    for poly in gs_targPoly:
        #Transform from lat long into the basemap x,y
        mmpoly = transform(mm, poly)
        patches.append(PolygonPatch(mmpoly))
    ax.add_collection(PatchCollection(patches,match_original=True, facecolor="#FFFFFF", alpha=0.2))
    
    #Platform Visualisation
    mm.scatter(plat_lon, plat_lat, marker='x', s=1200,label='plat', c="#000000")
    
    #Platform Label Visualisation - only possible in mpl, so a transformation must be done first
    platx, platy = mm(plat_lon, plat_lat)
    ax.annotate(plat_name, (platx, platy))

    #Platform Step-Out Radius Visualisation
    for i,r in enumerate(radii[1:],1): 
        circ_lonlat = GetCircumference([plat_lon, plat_lat],r , 51)
        circ_lon, circ_lat = list(zip(*circ_lonlat))    
        circ_x, circ_y = mm(circ_lon, circ_lat)
        ax.plot(circ_x, circ_y, dct_colors[len(radii)-i])

    #Well Concept Visualisaton
    mm.scatter(WC_lon, WC_lat, c=WC_col, s=100)
    
    #Well Conncept Label Visualisation
    for i, x in enumerate(WC_name):
        wcx, wcy = mm(WC_lon[i], WC_lat[i])
        ax.annotate(WC_name[i], ([wcx, wcy]), zorder=2)

#Example plot code for development/visualisation    
#WC_name, WC_points, WC_resource = UpdateWellConceptData(30, -60.34,-60.28,10.04,10.12, )
#c = ["#000000" for i in range(len(WC_name))]
#GenerateMap(WC_points, WC_name, WC_resource, c , -60.33,10.05,"Platform", -60.34,-60.28,10.04,10.12,[0,1.2,1.3],(14,22))    

In [22]:
def GenerateCreamingCurve(plat_x, plat_y, radii, figsize):
    '''Plots a creaming curve based on distance of Well Concepts from Platform, against radii
    '''
    #Subset the WCs to only those which are on the creaming curve
    WC_rank = CalculateRankOfWCs([(a[0], a[1]) for a in WC_points],(plat_x,plat_y),radii)

    #Construct dataframe of all well concepts
    df_WC = pd.DataFrame({"Well Concept": WC_name,"Resource": WC_resource, "Rank":WC_rank})
    
    #Reduce to df of well concepts on Creaming Curve (i.e. within the radii)
    df_WC_cc = df_WC[df_WC['Rank'] != 0].copy()
    
    #Map ranks to colors
    df_WC_cc.loc[:,'Color'] = df_WC_cc['Rank'].map(lambda x: dct_colors[x])
    
    
    df_WC_cc.sort_values(by="Resource", inplace=True, ascending=False)
    
    
    
    
    fig, ax = plt.subplots(figsize=figsize)    

    ax.bar(df_WC_cc['Well Concept'], df_WC_cc['Resource'], color=df_WC_cc['Color'])

    ax.set_xlabel("Well Concept Name")
    ax.set_ylabel("Resource")
    plt.show()

In [23]:
from ipywidgets import Button, Layout


In [24]:
lon_min = -60.34
lon_max = -60.28
lat_min = 10.06
lat_max = 10.12
n_concepts = 30

WC_name, WC_points, WC_resource = UpdateWellConceptData(n_concepts, lon_min, lon_max, lat_min, lat_max)

userInputs = [widgets.BoundedFloatText(
    value=round(lon_min +(random.random()*(lon_max-lon_min)), 3 ),
    min=lon_min,
    max=lon_max,
    step=0.001,
    description='Platform X:',
    disabled=False,
    layout=Layout(width='200px')
),
              
widgets.BoundedFloatText(
    value=round(lat_min +(random.random()*(lat_max-lat_min)), 3 ),
    min=lat_min,
    max=lat_max,
    step=0.001,
    description='Platform Y:',
    disabled=False,
    layout=Layout(width='200px')
),
              
  widgets.BoundedFloatText(
    value=1,
    min=0.01,
    max=1000,
    step=1,
    description='Radius1(km):',
    disabled=False,
    layout=Layout(width='180px')  
),
             
  widgets.BoundedFloatText(
    value=1.3,
    min=0.01,
    max=1000,
    step=1,
    description='Radius2(km):',
    disabled=False,
    layout=Layout(width='180px')
)]

In [25]:
#%matplotlib inline 
from IPython.display import display

output = widgets.Output(layout={'border': '1px solid black'})

def on_button_clicked(event):
    output.clear_output()

    with output:
        plat_x = userInputs[0].value
        plat_y = userInputs[1].value
        r1 = userInputs[2].value
        r2 = userInputs[3].value
        radii = [0, r1,r2]
        WC_colors = GetWCColors(WC_points,plat_x, plat_y, radii)
        
        GenerateMap(WC_points, WC_name, WC_resource, WC_colors, plat_x,plat_y,"platform", lon_min, lon_max, lat_min, lat_max,radii,  (16,14))
        GenerateCreamingCurve(plat_x, plat_y, radii, (16,8))
        
button = widgets.Button(
    description='click me',
    layout={'width': '300px'},
    button_style='success', 
    tooltip='Run',
    icon='refresh'
)

display(widgets.Box(userInputs), button)

button.on_click(on_button_clicked)

Box(children=(BoundedFloatText(value=-60.296, description='Platform X:', layout=Layout(width='200px'), max=-60…

Button(button_style='success', description='click me', icon='refresh', layout=Layout(width='300px'), style=But…

In [26]:
output

Output(layout=Layout(border='1px solid black'))

In [42]:
https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9

SyntaxError: invalid syntax (<ipython-input-42-116e7c3381ec>, line 1)

In [203]:
lon_min = -60.34
lon_max = -60.28
lat_min = 10.06
lat_max = 10.12
n_concepts = 30

WC_name, WC_points, WC_resource = UpdateWellConceptData(n_concepts, lon_min, lon_max, lat_min, lat_max)


In [204]:
import json
from bokeh.models.widgets import Slider

from bokeh.io import output_file, show, save

from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool, LabelSet,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
# Input GeoJSON source that contains features for plotting
from bokeh.models.markers import SquareCross


from bokeh.layouts import column, row, WidgetBox
from bokeh.models import Panel
from bokeh.models.widgets import Tabs

In [205]:
targ_source = GeoJSONDataSource(geojson = gdf_targPoly[['target_id', 'target_name', 'polygon']].to_json())




# Create figure object.
p = figure(title = 'Polygons', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset",
            x_range=(lon_min, lon_max),
            y_range=(lat_min, lat_max),          
          )
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Targets
p_targ = p.patches('xs','ys', source = targ_source,
                   fill_color = "lightskyblue",
                   line_color = "blue", 
                   line_width = 0.1, 
                   fill_alpha = 0.2)

# Well Concepts
wcx = [a[0] for a in WC_points]
wcy = [a[1] for a in WC_points]
source = ColumnDataSource({'x':wcx, 'y':wcy, 'name':WC_name, 'resource':WC_resource})

p.circle("x", "y", size=10, source=source,
         color='resource', line_color="black", fill_alpha=0.8)

labels = LabelSet(x="x", y="y", text="name", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)







# Create hover tool
p.add_tools(HoverTool(renderers = [p_targ],
                      tooltips = [('Target ID','@target_id'),
                                ('Target Name','@target_name')]))

#output_file("p.html")
#save(p)

In [206]:


lon_slider = Slider(start=lon_min, \
                    end=lon_max, \
                    value=round(lon_min +(random.random()*(lon_max-lon_min)), 3 ), \
                    step=0.01, \
                    title="Longitude")
lat_slider = Slider(start=lat_min, \
                    end=lat_max, \
                    value=round(lat_min +(random.random()*(lat_max-lat_min)), 3 ), \
                    step=0.01, \
                    title="Latitude")

#show(slider)



#show(p)

In [207]:
#Create LHS Control Panel
controls = WidgetBox(lon_slider, lat_slider)

# Create a row, with control panel and plot
layout = row(controls, p)


# Make a tab with the layout 
tab = Panel(child=layout, title = 'Geospatial PoC')
tabs = Tabs(tabs=[tab])
output_file("p.html")

save(tabs)

'/Users/Pri.balachandran@ibm.com/Desktop/BP/20191106_PlatformWCAnalysis/p.html'